## Loading the model
### Loading the model

In [1]:
import pickle
import numpy as np
import pandas as pd
import json
import os

In [2]:
model = pickle.load(open("model.pkl","rb"))
model

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('imputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                                 

Let's use Spotify Web API to get audio features of a new song and predicting its success with the trained model

In [4]:
# Reading Spotify web API credentials from settings.env hidden file

with open('../spotify_api/settings.env') as f:
    env_vars = json.loads(f.read())

In [5]:
# Set environment variables
os.environ['SPOTIPY_CLIENT_ID'] = env_vars['SPOTIPY_CLIENT_ID']
os.environ['SPOTIPY_CLIENT_SECRET'] = env_vars['SPOTIPY_CLIENT_SECRET']

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

query = 'you oughta know'

search = sp.search(q = query, type='track', market='US')

#print('artist: ',search['tracks']['items'][0]['artists'][0]['name'], 
#      '\nsong: ',search['tracks']['items'][0]['name'],
#      '\nalbum: ',search['tracks']['items'][0]['album']['name'],
#      '\ntrack_id: ',search['tracks']['items'][0]['id'])

In [7]:
print('artist: ',search['tracks']['items'][0]['artists'][0]['name'], 
      '\nsong: ',search['tracks']['items'][0]['name'],
      '\nalbum: ',search['tracks']['items'][0]['album']['name'],
      '\ntrack_id: ',search['tracks']['items'][0]['id'])

track_id = search['tracks']['items'][0]['id']

artist:  Alanis Morissette 
song:  You Oughta Know - 2015 Remaster 
album:  Jagged Little Pill (2015 Remaster) 
track_id:  3jS7bB0oXVOwGFZn3aE5NV


In [8]:
audio_features = sp.audio_features(track_id)
audio_features

[{'danceability': 0.665,
  'energy': 0.834,
  'key': 4,
  'loudness': -7.737,
  'mode': 1,
  'speechiness': 0.0576,
  'acousticness': 0.21,
  'instrumentalness': 0,
  'liveness': 0.452,
  'valence': 0.411,
  'tempo': 105.292,
  'type': 'audio_features',
  'id': '3jS7bB0oXVOwGFZn3aE5NV',
  'uri': 'spotify:track:3jS7bB0oXVOwGFZn3aE5NV',
  'track_href': 'https://api.spotify.com/v1/tracks/3jS7bB0oXVOwGFZn3aE5NV',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3jS7bB0oXVOwGFZn3aE5NV',
  'duration_ms': 249493,
  'time_signature': 4}]

In [15]:
X = pd.json_normalize(audio_features[0])

In [16]:
type(X)

pandas.core.frame.DataFrame

In [17]:
X

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.665,0.834,4,-7.737,1,0.0576,0.21,0,0.452,0.411,105.292,audio_features,3jS7bB0oXVOwGFZn3aE5NV,spotify:track:3jS7bB0oXVOwGFZn3aE5NV,https://api.spotify.com/v1/tracks/3jS7bB0oXVOw...,https://api.spotify.com/v1/audio-analysis/3jS7...,249493,4


We apply the same transformations as when we trained the model

In [20]:
X_clean = X.drop(['type', 'id', 'uri', 'track_href', 'analysis_url'], axis=1)

In [21]:
X_clean

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.665,0.834,4,-7.737,1,0.0576,0.21,0,0.452,0.411,105.292,249493,4


Now let's find the prediction for the track with those features

In [23]:
print(model.predict(X_clean))

[1.]


The result for 'You Oughta Know', a hit track from 1996, is **1, successful**.

Let's predict the success of a hit song that is already in the model

In [31]:
# Trying to condense the query in one search line

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

query = 'macarena'

audio_features = sp.audio_features((sp.search(q = query, type='track', market='US'))['tracks']['items'][0]['id'])
X = pd.json_normalize(audio_features[0])
X_clean = X.drop(['type', 'id', 'uri', 'track_href', 'analysis_url'], axis=1)
print(model.predict(X_clean))

[1.]


### Search with input dialog

In [53]:
import pickle
import numpy as np
import pandas as pd
import json
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import altair as alt

model = pickle.load(open("model.pkl","rb"))

with open('../spotify_api/settings.env') as f:
    env_vars = json.loads(f.read())
os.environ['SPOTIPY_CLIENT_ID'] = env_vars['SPOTIPY_CLIENT_ID']
os.environ['SPOTIPY_CLIENT_SECRET'] = env_vars['SPOTIPY_CLIENT_SECRET']

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

print('Enter song name or artist + song name:')

query = input()

audio_features = sp.audio_features((sp.search(q = query, type='track', market='US'))['tracks']['items'][0]['id'])

#audio_features = sp.audio_features('549DBe4tw3IYPnedaz0q0J')

X = pd.json_normalize(audio_features[0])

X_clean = X.drop(['type', 'id', 'uri', 'track_href', 'analysis_url'], axis=1)

if (model.predict(X_clean)[0])==0:
    print('NOT HOT\nProbability: %.0f' % ((model.predict_proba(X_clean)[0][0])*100), '%')
else:
    print('HOT!!!\nProbability: %.0f' % ((model.predict_proba(X_clean)[0][1])*100), '%')
#print(model.predict_proba(X_clean))

track = sp.track( X['id'][0] )

print('Artist: ', pd.json_normalize(track)['artists'][0][0]['name'])
print('Song:   ', pd.json_normalize(track)['name'][0])
print('Album:  ', pd.json_normalize(track)['album.name'][0])
print('Date:   ', pd.json_normalize(track)['album.release_date'][0][0:4])
print(pd.json_normalize(track)['album.images'][0][1]['url'])
print(pd.json_normalize(track)['preview_url'][0])

Enter song name or artist + song name:
rosalia malamente
HOT!!!
Probability: 88 %
Artist:  ROSALÍA
Song:    MALAMENTE - Cap.1: Augurio
Album:   El Mal Querer
Date:    2018
https://i.scdn.co/image/ab67616d00001e02f4d64a6a6b7e24b6bd9f009f
https://p.scdn.co/mp3-preview/c3eb3a02bd252e22b93f87af33b7e8ce94ac8c9f?cid=501b4799403a409f902c689964bf9f91
